In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import fitz
import json
from google.colab import files
from transformers import pipeline
from pydantic import BaseModel, Field
from typing import List, Dict
from itertools import islice

2025-05-26 10:43:54.793770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748256234.967561      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748256235.019384      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
#!pip uninstall -y torch torchvision torchaudio
!pip install --quiet transformers accelerate pymupdf pydantic
!pip install marker-pdf
!pip install torch torchvision torchaudio

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
timm 1.0.15 requires torchvision, which is not installed.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 99.3 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.


In [5]:

from huggingface_hub import notebook_login
notebook_login()

In [6]:
# 4. Load the Model ( any fuckin' model )

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct" 

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

eq_extractor = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    temperature=0.0,
    do_sample=False
)


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Utility: chunk an iterable into fixed-size chunks
def chunked_text(text, size=500):
    words = text.split()
    for i in range(0, len(words), size):
        yield " ".join(words[i:i+size])


In [21]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

PDF_PATH = "/kaggle/input/same-shit/next-generation-sample-questions-advanced-algebra-and-functions.pdf"
converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter(PDF_PATH)
markdown_text = text_from_rendered(rendered)[0]

# 7. Chunk Markdown text
#chunks = []
#for idx, chunk in enumerate(chunked_text(markdown_text, size=500)):
 #   chunks.append({"page": idx+1, "markdown": chunk})

Running OCR Error Detection: 100%|██████████| 1/1 [00:00<00:00, 26.15it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing Text: 100%|██████████| 2/2 [00:03<00:00,  1.99s/it]
Detecting bboxes: 0it [00:00, ?it/s]


In [23]:
# to show off some pages 
print("=== Raw Markdown Preview ===")
print(markdown_text[:5000])  

# Check for equation remnants
import re
potential_equations = re.findall(r'\$.*?\$|\\\(.*?\\\)', markdown_text)
print(f"\\nFound {len(potential_equations)} potential equations in raw text:")
print(potential_equations[:50])  # Show the candidates

=== Raw Markdown Preview ===
![](_page_0_Picture_0.jpeg)

# Advanced Algebra and Functions

**Sample Questions**

## **College Board**

College Board is a mission-driven not-for-profit organization that connects students to college success and opportunity. Founded in 1900, College Board was created to expand access to higher education. Today, the membership association is made up of over 6,000 of the world's leading education institutions and is dedicated to promoting excellence and equity in education. Each year, College Board helps more than seven million students prepare for a successful transition to college through programs and services in college readiness and college success—including the SAT® and the Advanced Placement® Program. The organization also serves the education community through research and advocacy on behalf of students, educators, and schools.

For further information, visit **[collegeboard.org](www.collegeboard.org)**.

## **ACCUPLACER Advanced Algebra and Functio

In [15]:
def make_prompt(chunk_md):
    return '''You are an expert mathematical text extractor. Your task is to accurately identify and extract ONLY genuine mathematical equations from the provided academic content, which is formatted in Markdown.

Task Instructions:
  1. Identification:
     • Extract only valid mathematical equations or expressions (e.g., integrals, summations, matrices, fractions).
     • Do NOT extract plain text descriptions, code blocks, or non-equation content.
  2. Conversion:
     • Convert each extracted equation into syntactically correct LaTeX, using proper math mode delimiters ($...$, \\[...\\], or \begin{equation}...\end{equation}).
  3. Integrity:
     • Do NOT generate or hallucinate any equations not present in the text.
     • Preserve all original symbols, indices, exponents, and formatting as closely as possible.
  4. Output Format:
     • Return the result as a JSON array of strings, where each string is one LaTeX-formatted equation.
     • If no equations are found, return an empty list: [].

Context:
  • The input is a Markdown chunk converted from a PDF page, so it may include headings, lists, and inline formatting.
  • Focus solely on equations; do NOT include any explanatory text or observations in the output.

Markdown Input:

{chunk_md}

JSON Output:
'''

In [16]:
results = []
for idx, block in enumerate(chunks, start=1):
    prompt = make_prompt(block['markdown'])
    output = eq_extractor(prompt)[0]['generated_text']
    try:
        json_start = output.index('[')
        equations = json.loads(output[json_start:])
    except Exception:
        equations = []
    results.append({
        'chunk_id': idx,
        'page': block['page'],
        'equations': equations
    })

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` 

In [17]:
# 9. Save results to JSON file
with open('equations_extracted.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"Extraction complete. {len(results)} chunks processed. Results saved to equations_extracted.json.")

Extraction complete. 9 chunks processed. Results saved to equations_extracted.json.


In [20]:
from IPython.display import FileLink


FileLink(r'equations_extracted.json')


/kaggle/working/equations_extracted.json

In [18]:
print("\n=== Extracted Equations ===\n")
for page in results:
    print(f"Page {page['page']}:")
    if page['equations']:
        for eq in page['equations']:
            print(f"  - {eq}")
    else:
        print("  No equations found.")



=== Extracted Equations ===

Page 1:
  No equations found.
Page 2:
  No equations found.
Page 3:
  No equations found.
Page 4:
  No equations found.
Page 5:
  No equations found.
Page 6:
  No equations found.
Page 7:
  No equations found.
Page 8:
  No equations found.
Page 9:
  No equations found.
